In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader

In [3]:
transforms={'train':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),
    'val':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [19]:
data_dir='F:/DL/venv/hymenoptera_data'
#image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),transform=transforms[x]) for x in ['train','val']}


In [20]:
image_datasets={}
image_datasets['train']=datasets.ImageFolder(os.path.join(data_dir,'train'),transform=transforms['train'])
image_datasets['val']=datasets.ImageFolder(os.path.join(data_dir,'val'),transform=transforms['val'])

In [21]:
data_loader={}
data_loader['train']=DataLoader(image_datasets['train'],batch_size=32,shuffle=True)
data_loader['val']=DataLoader(image_datasets['val'],batch_size=32,shuffle=True)

In [29]:
model=models.resnet18(pretrained=True)
#print(model)
model_ftrs=model.fc.in_features
model.fc = nn.Sequential()
model.fc.ft1=nn.Linear(model_ftrs,out_features=256)
model.fc.ft2=nn.Linear(256,out_features=64)
model.fc.ft3=nn.Linear(64,out_features=16)
model.fc.ft4=nn.Linear(16,out_features=2)

criterian=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.fc.parameters(),lr=0.0001)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [30]:
for epoch in range(30):
    for input,label in data_loader['train']:
        output=model(input)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(loss.item())


0.5822532773017883
0.542256236076355
0.509911060333252
0.39172232151031494
0.5359135866165161
0.45785778760910034
0.41510218381881714
0.36795946955680847
0.4718874394893646
0.25722014904022217
0.39034217596054077
0.378290057182312
0.4069424271583557
0.18685218691825867
0.28847867250442505
0.15312369167804718
0.28245407342910767
0.18787604570388794
0.17281298339366913
0.4511038362979889
0.3600448668003082
0.17439064383506775
0.3270472586154938
0.07958986610174179
0.24569246172904968
0.42311644554138184
0.09057775884866714
0.18626657128334045
0.11880271136760712
0.25350385904312134


In [31]:
model.eval()
with torch.no_grad():
    correct=0
    total=0
    for input,label in data_loader['val']:
        output=model(input)
        _,predicted=torch.max(output.data,1)
        total+=label.size(0)
        correct+=(predicted==label).sum().item()
    print('Accuracy: {}%'.format(100*correct/total))
    print(total, correct)

Accuracy: 87.58169934640523%
153 134
